### Here we try to predict if two person would result a match and give some importance featurues

Load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
data_df = pd.read_csv("../input/Speed Dating Data.csv", encoding="ISO-8859-1")
data_df.head()

There are Nan values in both 'field' and 'field_cd', but there are some entries that 'field' is filled but with 'filed_cd' empty, so we convert them here

In [ ]:
print("Numer of ppl did not fill field:", sum(pd.isnull(data_df['field'])))
print("Numer of ppl did not fill field_cd:", sum(pd.isnull(data_df['field_cd'])))
print("Number of ppl did not fill career:", sum(pd.isnull(data_df['career'])))

# convert all filled field to field code
f_fcd = data_df[['field','field_cd']].drop_duplicates() # get all listed filed name and its code
nan_ind = pd.isnull(f_fcd).any(1).nonzero()[0] # row has Nan
f_fcd.drop(f_fcd.index[nan_ind],inplace=True) # remove rows that has Nan

fcd_ind = pd.isnull(data_df['field_cd']).nonzero()[0] # row where 'field_cd' is Nan
data_ind = data_df.index
n = 0
for i in fcd_ind:
    field_i = data_df.loc[data_ind[i], 'field']
    f_cd = f_fcd[f_fcd['field']==field_i]['field_cd'].values
    if pd.isnull(f_cd) == 0: # if the person did not leave 'field' empty
        n += 1
        data_df.loc[data_ind[i], 'field_cd'] = f_cd[0]
print("Done converting with {} filed_cd added!".format(n))

We will only use some relevant and avaliable features

In [ ]:
# note 'career_c' is incomplete and needs to add
# And also 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1' are not filled 
use_features = ['iid', 'gender', 'pid', 'match', 'samerace', 'age_o', 'race_o', \
                 'pf_o_att', 'pf_o_sin', 'pf_o_int','pf_o_fun', 'pf_o_amb', 'pf_o_sha',\
                 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'sports',\
                 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',\
                 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy',\
                 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr2_1', 'sinc2_1',\
                 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1','fun3_1', 'intel3_1', 'amb3_1']
Data_df = data_df[use_features]

build the vector that contains the information both for the male and female

In [ ]:
# first get rid off any rows contain Nan value
inds = pd.isnull(Data_df).any(1).nonzero()[0] # row index that contains Nan
print("number of rows contain Nan:", len(inds))
Data_df= Data_df.drop(Data_df.index[inds])
# notice pid is float, so we change it to int
Data_df['pid'] = Data_df['pid'].astype(int)
Data_df['iid'] = Data_df['iid'].astype(int)

mdata_df = Data_df[Data_df['gender']==1]
fdata_df = Data_df[Data_df['gender']==0]
print(mdata_df.shape)
print(fdata_df.shape)
same1 = []
for i in mdata_df.pid.values:
    if i not in fdata_df.iid.values:
        same1.append(i)
print("some guy's partener is not found in fdata:", list(set(same1)))
same2 = []
for i in fdata_df.pid.values:
    if i not in mdata_df.iid.values:
        same2.append(i)
print("some girl's partener is not found in mdata:", list(set(same2)))

# male features in combination
cmfeatures = ['iid', 'pid', 'match', 'samerace', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out',\
             'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',\
             'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy',\
             'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr2_1', 'sinc2_1',\
             'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1','fun3_1', 'intel3_1', 'amb3_1']
new_mdata = mdata_df[cmfeatures]
# female features in combination
cffeatures = [cmfeatures[0]] + cmfeatures[4:]
new_fdata = fdata_df[cffeatures].drop_duplicates()
new_data = pd.DataFrame.copy(new_mdata)
new_fdata.columns = [i+'_f' for i in cffeatures] # rename the feature name of female

In [ ]:
df = pd.DataFrame()
for i in new_mdata.index.values:
    m_info = new_mdata.loc[i,:]
    pid = new_mdata.loc[i,'pid'] # this is the (female)partener's ID
    f_ind = new_fdata.iid_f==pid
    if sum(f_ind) !=0: # append only if the pid is found in female iid
        f_info_df = new_fdata[f_ind]
        f_info = f_info_df.loc[f_info_df.index[0],:]
        combined = m_info.append(f_info)
        df = df.append(combined,ignore_index=True)
# now drop features that we do not need for prediction
drop_features = ['iid', 'pid', 'iid_f']
print("Done making data for pairs")
pair_df = df.drop(drop_features,axis=1)
pair_df.head()

Now let us build the model, first, and see the number of each class(0 = no, 1 = yes)

In [ ]:
pair_train = pair_df.drop('match',axis=1)
pair_label = pair_df['match']
print("total training data size:", pair_df.shape)
pd.Series.value_counts(pair_label)

we will use a tuned xgb model

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV   #Performing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 22, 4

from time import time

# create train and test data
train_data, test_data = train_test_split(pair_df, test_size=0.1, random_state=42, stratify=pair_df['match'])
predictors = [x for x in pair_df.columns if x not in ['match']]
print("train shape:", train_data.shape)
print("test shape:", test_data.shape)

### Train our model!

(see details :http://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

In [ ]:
def modelfit(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=20):
    t1 = time()
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain['match'].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics=['auc'], early_stopping_rounds=early_stopping_rounds, stratified=True)#, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['match'],eval_metric=['auc'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    #Predict test set:
    dtest_predictions = alg.predict(dtest[predictors])
    dtest_predprob = alg.predict_proba(dtest[predictors])[:,1]
    
    t2 = time()    
    #Print model report:
    print("\nModel Report (took {0:.2f}sec)".format(t2-t1))
    print("The result params is:\n", alg.get_xgb_params())
    print("Train Accuracy: {0:.2f}%".format(metrics.accuracy_score(dtrain['match'].values, dtrain_predictions)))
    print("Train AUC Score: {0:.4f}".format(metrics.roc_auc_score(dtrain['match'], dtrain_predprob)))
    print("Test Accuracy: {0:.2f}%".format(metrics.accuracy_score(dtest['match'].values, dtest_predictions)))
    print("Test AUC Score: {0:.4f}".format(metrics.roc_auc_score(dtest['match'], dtest_predprob)))
    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    return dtest_predprob, feat_imp

In [ ]:
clf = XGBClassifier(learning_rate =0.01,
                     n_estimators=5000,
                     max_depth=7,
                     min_child_weight=12,
                     gamma=0.,
                     subsample=0.85,
                     colsample_bytree=0.85,
                     reg_alpha=0.,
                     objective= 'binary:logistic',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27)
result = modelfit(clf, train_data, test_data, predictors)

First 10 important features: (importance in descending order)

In [ ]:
print(result[1].index.values[:10])

Least 10 important features: (importance in ascending order)

In [ ]:
print(list(result[1].index.values[-10:])[::-1])

### To evaluate how good our model is, plot ROC

In [ ]:
from sklearn.metrics import roc_curve, auc
rcParams['figure.figsize'] = 10, 8
false_positive_rate, true_positive_rate, thresholds = roc_curve(test_data['match'], result[0])
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.plot([0,1],[0,1],'r--')
plt.legend(loc='lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Although our model does predict the outcome of the match, but AUC score is 0.768 which means our model is not the best yet or fair.(see more about auc_roc: https://datamize.wordpress.com/2015/01/24/how-to-plot-a-roc-curve-in-scikit-learn/ and http://gim.unmc.edu/dxtests/roc3.htm)